## Import the libraries

In [1]:
import sys
import numpy as np

sys.path.append('/home/nexus-admin/NEXUS_RF/BackendTools')
import MB_equations as MBe
import TimestreamHelperFunctions as Thf

## Define some parameters

In [2]:
## MB Results come from fitting a temperature scan at a specified RF power
use_nexus_MBvals = False

if use_nexus_MBvals:
    ## As measured in NEXUS
    MB_fit_vals = np.array([4.24216+9.75e-6, 
                            0.177  ,
                            0.0125 ,
                            4.1e7  ]) ## [ F0 [GHz] , Delta [meV] , alpha , Qi0 ]
else:
    ## As measured on Caltech sister device (https://arxiv.org/pdf/2111.08064.pdf)
    MB_fit_vals = np.array([4.24201000, 
                            0.184     ,
                            0.03801   ,
                            4.05538e5 ]) ## [ F0 [GHz] , Delta [meV] , alpha , Qi0 ]
    
## How much attenuation is in the lines before the chip
line_atten_dB = 56.5

## Define the MB parameters
alpha = MB_fit_vals[2]
Delta = MB_fit_vals[1] ## meV
D     = Delta*1e-3     ##  eV

## Temperature of the device
T = 10.0e-3 ## K

## Superconducting volume per resonator
V = 3e4 ## um^3

## Energy of the incident photons
hv = 2.64 ## eV

## Get the readout power and frequency

In [3]:
datapath = '/data/USRP_Laser_Data'
series   = "20221206_105445"

sum_file, dly_file, vna_file, nse_files, led_files = Thf.GetFiles(series, 
                                                        base_path=datapath,
                                                        sep_noise_laser=True,
                                                        verbose=False)

md, charFs, charZs = Thf.UnpackSummary(sum_file, verbose=False)
rf_power = md['power'] ## dBm at USRP
f        = charFs[1]   ## MHz
print("RF Power at USRP:", rf_power, "dBm")

RF Power at USRP: -30 dBm


/home/nexus-admin/NEXUS_RF/BackendTools/TimestreamHelperFunctions.py:108: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  l_dict[kk] = md[k][kk].value


## Calculate $\kappa_2$

In [4]:
k2 = MBe.kappa_2(T, f*1e6, D) *1e6*1e6*1e6

## Display the parameters

In [5]:
print("Readout Power:", rf_power, "dBm")
print("Readout Freq.:", f       , "MHz")
print("Device Temp.: ", T*1e3   , "mK")
print("==============================")
print("alpha: ", alpha*100, "%")
print("Delta: ", Delta    , "meV")
print("V:     ", V        , "um^3")
print("k2:    ", k2       , "um^-3")

Readout Power: -30 dBm
Readout Freq.: 4242.046627 MHz
Device Temp.:  10.0 mK
alpha:  3.801 %
Delta:  0.184 meV
V:      30000.0 um^3
k2:     3.913144945324984e-07 um^-3


## Calculate values for NEXUS

In [6]:
## How many resonators per device
M_NEXUS    = 1
s_df_NEXUS = 1.9e-7

### Equation 4 (and 14)

In [7]:
## Calculate photon shot noise prefactor (x sqrt(N) eta_ph)
sigma_LED_df = (1/M_NEXUS) * (alpha * k2 / V) * ( hv / (2.*D) )
print("sigma_LED_df =",sigma_LED_df,"x sqrt(N) x eta_ph")

sigma_LED_df = 3.5567935501952808e-09 x sqrt(N) x eta_ph


### Equation 8 (and 15)

In [8]:
## Calculate qp density corresponding to a specific pulse height
n_qp       = 2 * s_df_NEXUS / (alpha *k2)
print("n_qp =",n_qp,"qp/um^3")

n_qp = 25.54816970256883 qp/um^3


### Equation 6 (and 16)

In [9]:
## Calculate how many photons correpsond to that qp density (x 1/eta_ph)
N_gamma = n_qp * M_NEXUS * V * D / (hv)
print("N_gamma =",N_gamma,"/eta_ph")

N_gamma = 53.41890028718937 /eta_ph


### Equation 9 (and 17)

In [10]:
## Calculate photon shot noise prefactor (x sqrt(eta_ph))
sigma_LED_df = np.sqrt( (alpha*k2/2.) * s_df_NEXUS * (hv/(M_NEXUS*V*D)) )
print("sigma_LED_df =",sigma_LED_df,"x sqrt(eta_ph)")

sigma_LED_df = 2.5995976122029028e-08 x sqrt(eta_ph)


### Equation 19

In [11]:
## Set a limit on eta_ph (x f^2)
limit = np.power( s_df_NEXUS / sigma_LED_df ,2)
print("eta_ph <",limit,"f^2")
print("For f=0.60%, eta_ph <", limit*np.power( (0.60/100.) ,2), )
print("For f=1.05%, eta_ph <", limit*np.power( (1.05/100.) ,2), )
print("For f=3.45%, eta_ph <", limit*np.power( (3.45/100.) ,2), )
print("For f=4.30%, eta_ph <", limit*np.power( (4.30/100.) ,2), )

eta_ph < 53.41890028718938 f^2
For f=0.60%, eta_ph < 0.0019230804103388178
For f=1.05%, eta_ph < 0.00588943375666263
For f=3.45%, eta_ph < 0.06358184606682717
For f=4.30%, eta_ph < 0.09877154663101315


In [12]:
0.6/0.06358184606682717

9.436655855656896

In [13]:
n_qp * (3.45/100.)

0.8814118547386247

In [14]:
100 * (0.60/100.)

0.6

In [15]:
np.sqrt( (100./25.)*(80./1.) )

17.88854381999832

In [16]:
np.sqrt( (100./n_qp)*(80./M_NEXUS) )

17.695592039399184

In [17]:
(M_NEXUS/80.)*18

0.225

In [18]:
(M_NEXUS/80.)*np.sqrt( (100./25.)*(80./1.) )

0.223606797749979

In [19]:
(M_NEXUS/80.)*np.sqrt( (100./n_qp)*(80./M_NEXUS) )

0.2211949004924898